In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [3]:
# 10 class problem
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [4]:
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch7/'

# Read the image. Convert them from Python to Tensor. And then N(0,1). 
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [5]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

# Defined a transformation function from the original input
print(cifar10)

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ../data-unversioned/p1ch7/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.4915, 0.4823, 0.4468), std=(0.247, 0.2435, 0.2616))
           )


In [17]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10 
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

# Only the two labels (it's good!)

In [19]:
#
# Oh, hello there. In order to call the model, we need to make the input
# have the right dimensions. We recall that our model expects 3,072 features
# in the input, and that nn works with data organized into batches along the
# zeroth dimension. So we need to turn our 3 × 32 × 32 image into a 1D tensor
# and then add an extra dimension in the zeroth position. We learned how to do
# this in chapter 3:
#

import torch.nn as nn

model = nn.Sequential(
             nn.Linear(3072, 512),
             nn.Tanh(),
             nn.Linear(512, 2),
             nn.LogSoftmax(dim=1))

In [20]:
loss = nn.NLLLoss()

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
 
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True)
 
model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 2),
            nn.LogSoftmax(dim=1))
 
learning_rate = 1e-2
 
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
 
loss_fn = nn.NLLLoss()
 
n_epochs = 100
 
for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
 
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
    
# At each inner iteration, imgs is a tensor of size 64 × 3 × 32 × 32--that is,
# a minibatch of 64 (32 × 32) RGB images--while labels is a tensor of size 64
# containing label indices.

Epoch: 0, Loss: 0.353794
Epoch: 1, Loss: 0.447301
Epoch: 2, Loss: 0.513703
Epoch: 3, Loss: 0.579145
Epoch: 4, Loss: 0.413433
Epoch: 5, Loss: 0.422155
Epoch: 6, Loss: 0.258638
Epoch: 7, Loss: 0.334352
Epoch: 8, Loss: 0.363473
Epoch: 9, Loss: 0.502085
Epoch: 10, Loss: 0.389964
Epoch: 11, Loss: 0.301984
Epoch: 12, Loss: 0.448011
Epoch: 13, Loss: 0.181602
Epoch: 14, Loss: 0.327986
Epoch: 15, Loss: 0.356176
Epoch: 16, Loss: 0.285281
Epoch: 17, Loss: 0.403240
Epoch: 18, Loss: 0.436240
Epoch: 19, Loss: 0.290025
Epoch: 20, Loss: 0.324853
Epoch: 21, Loss: 0.441826
Epoch: 22, Loss: 0.233857
Epoch: 23, Loss: 0.428139
Epoch: 24, Loss: 0.313984
Epoch: 25, Loss: 0.210950
Epoch: 26, Loss: 0.292895
Epoch: 27, Loss: 0.254778
Epoch: 28, Loss: 0.179586
Epoch: 29, Loss: 0.178726
Epoch: 30, Loss: 0.196208
Epoch: 31, Loss: 0.088738
Epoch: 32, Loss: 0.319536
Epoch: 33, Loss: 0.325243
Epoch: 34, Loss: 0.101193
Epoch: 35, Loss: 0.113454
Epoch: 36, Loss: 0.053813
Epoch: 37, Loss: 0.204897
Epoch: 38, Loss: 0.077

In [26]:
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)
 
correct = 0
total = 0
 
with torch.no_grad():
    for imgs, labels in val_loader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
 
print("Accuracy: %f", correct / total)

Accuracy: %f 0.801


In [27]:
model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 2))
 
learning_rate = 1e-2
 
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
 
loss_fn = nn.CrossEntropyLoss()
 
n_epochs = 100
 
for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
 
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
    
#
# Note that the numbers will be exactly the same as with nn.LogSoftmax and nn.NLLLoss.
# It’s just more convenient to do it all in one pass, with the only gotcha being that the
# output of our model will not be interpretable as probabilities (or log probabilities).
# We’ll need to explicitly pass the output through a softmax to obtain those.
#

Epoch: 0, Loss: 0.418254
Epoch: 1, Loss: 0.518168
Epoch: 2, Loss: 0.390147
Epoch: 3, Loss: 0.474446
Epoch: 4, Loss: 0.774831
Epoch: 5, Loss: 0.498739
Epoch: 6, Loss: 0.466359
Epoch: 7, Loss: 0.372523
Epoch: 8, Loss: 0.417615
Epoch: 9, Loss: 0.240815
Epoch: 10, Loss: 0.148146
Epoch: 11, Loss: 0.289371
Epoch: 12, Loss: 0.212687
Epoch: 13, Loss: 0.184523
Epoch: 14, Loss: 0.477757
Epoch: 15, Loss: 0.247631
Epoch: 16, Loss: 0.374469
Epoch: 17, Loss: 0.221565
Epoch: 18, Loss: 0.279901
Epoch: 19, Loss: 0.399121
Epoch: 20, Loss: 0.519400
Epoch: 21, Loss: 0.226285
Epoch: 22, Loss: 0.209948
Epoch: 23, Loss: 0.338107
Epoch: 24, Loss: 0.309127
Epoch: 25, Loss: 0.220578
Epoch: 26, Loss: 0.113648
Epoch: 27, Loss: 0.263938
Epoch: 28, Loss: 0.141311
Epoch: 29, Loss: 0.558881
Epoch: 30, Loss: 0.068996
Epoch: 31, Loss: 0.107059
Epoch: 32, Loss: 0.158684
Epoch: 33, Loss: 0.190719
Epoch: 34, Loss: 0.176785
Epoch: 35, Loss: 0.485824
Epoch: 36, Loss: 0.273315
Epoch: 37, Loss: 0.080837
Epoch: 38, Loss: 0.231

In [28]:
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)
 
correct = 0
total = 0
 
with torch.no_grad():
    for imgs, labels in val_loader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
 
print("Accuracy: %f", correct / total)

Accuracy: %f 0.8145
